In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import statsmodels.api as sm

/anaconda3/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data=pd.read_csv("yijiashanchu01.csv")
data.head()   
#增加赔率变量
data["winchg"]=data["chuwin"]-data["win"]
data["drawchg"]=data["chudraw"]-data["draw"]
data["losechg"]=data["chulose"]-data["lose"]
data["chuwin2"]=data["chuwin"]**2
data["chudraw2"]=data["chudraw"]**2
data["chulose2"]=data["chulose"]**2
#增加常数项
data["intercept"]=1.0
data.head()

,Unnamed: 0,chuwin,chudraw,chulose,win,draw,lose,reslut,chuwinp,chudrawp,...,drawp,losep,"result1,0",winchg,drawchg,losechg,chuwin2,chudraw2,chulose2,intercept
0,0,4.5,7.50,1.65,4.1,5.75,1,0,0.222222,0.133333,...,0.173913,1.000000,0,0.4,1.75,0.65,20.25,56.2500,2.7225,1.0
1,1,3.9,5.75,1.55,4.2,6.00,3,2,0.256410,0.173913,...,0.166667,0.333333,1,-0.3,-0.25,-1.45,15.21,33.0625,2.4025,1.0
2,2,3.9,5.75,1.55,4.2,6.00,3,2,0.256410,0.173913,...,0.166667,0.333333,1,-0.3,-0.25,-1.45,15.21,33.0625,2.4025,1.0
3,3,4.0,5.50,1.65,4.0,5.00,1,1,0.250000,0.181818,...,0.200000,1.000000,0,0.0,0.50,0.65,16.00,30.2500,2.7225,1.0
4,4,3.8,5.75,1.72,3.6,5.00,2,2,0.263158,0.173913,...,0.200000,0.500000,1,0.2,0.75,-0.28,14.44,33.0625,2.9584,1.0


In [3]:
data.columns

Index(['Unnamed: 0', 'chuwin', 'chudraw', 'chulose', 'win', 'draw', 'lose',
       'reslut', 'chuwinp', 'chudrawp', 'chulosep', 'winp', 'drawp', 'losep',
       'result1,0', 'winchg', 'drawchg', 'losechg', 'chuwin2', 'chudraw2',
       'chulose2', 'intercept'],
      dtype='object')

In [4]:
train_cols=data.columns[[8,9,10,15,16,17,18,19,20,21]]
logit=sm.Logit(data["result1,0"], data[train_cols])

result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.398812
         Iterations 7


In [5]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              result1,0   No. Observations:                 3594
Model:                          Logit   Df Residuals:                     3584
Method:                           MLE   Df Model:                            9
Date:                Sun, 24 Dec 2017   Pseudo R-squ.:                  0.4020
Time:                        12:52:49   Log-Likelihood:                -1433.3
converged:                       True   LL-Null:                       -2396.8
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
chuwinp      -11.9025      4.417     -2.695      0.007     -20.560      -3.245
chudrawp      -1.2699      1.928     -0.659      0.510      -5.049       2.509
chulosep      -4.7739      2.387     -2.000      0.046      -9.453      -0.095
winchg         0.0334      0.272      0.123      0.902      -0.500       0.567
drawchg       -0.2804      0.167     -1.683      0.092      -0.607       0.046
losechg       -1.7468      0.061    -28.680      0.000      -1.866      -1.627
chuwin2       -0.1663      0.071     -2.353      0.019      -0.305      -0.028
chudraw2       0.0158      0.013      1.261      0.207      -0.009       0.040
chulose2       0.1408      0.016      8.599      0.000       0.109       0.173
intercept      8.2645      2.836      2.914      0.004       2.706      13.823
==============================================================================
"""

In [7]:
# 构建预测集
# 与训练集相似，一般也是通过 pd.read_csv() 读入
# 在这边为方便，我们将训练集拷贝一份作为预测集（不包括 admin 列）
import copy
combos = copy.deepcopy(data)
# 数据中的列要跟预测时用到的列一致
predict_cols = combos.columns[[8,9,10,15,16,17,18,19,20,21]]
 
# 进行预测，并将预测评分存入 predict 列中
combos['predict'] = result.predict(combos[predict_cols])

In [8]:
combos.head(20)

,Unnamed: 0,chuwin,chudraw,chulose,win,draw,lose,reslut,chuwinp,chudrawp,...,losep,"result1,0",winchg,drawchg,losechg,chuwin2,chudraw2,chulose2,intercept,predict
0,0,4.50,7.50,1.65,4.10,5.75,1,0,0.222222,0.133333,...,1.000000,0,0.40,1.75,0.65,20.2500,56.2500,2.7225,1.0,0.239994
1,1,3.90,5.75,1.55,4.20,6.00,3,2,0.256410,0.173913,...,0.333333,1,-0.30,-0.25,-1.45,15.2100,33.0625,2.4025,1.0,0.944538
2,2,3.90,5.75,1.55,4.20,6.00,3,2,0.256410,0.173913,...,0.333333,1,-0.30,-0.25,-1.45,15.2100,33.0625,2.4025,1.0,0.944538
3,3,4.00,5.50,1.65,4.00,5.00,1,1,0.250000,0.181818,...,1.000000,0,0.00,0.50,0.65,16.0000,30.2500,2.7225,1.0,0.286722
4,4,3.80,5.75,1.72,3.60,5.00,2,2,0.263158,0.173913,...,0.500000,1,0.20,0.75,-0.28,14.4400,33.0625,2.9584,1.0,0.722728
5,7,3.80,7.50,1.57,4.20,5.75,1,1,0.263158,0.133333,...,1.000000,0,-0.40,1.75,0.57,14.4400,56.2500,2.4649,1.0,0.322093
6,8,4.50,5.75,1.55,4.33,5.75,2,2,0.222222,0.173913,...,0.500000,1,0.17,0.00,-0.45,20.2500,33.0625,2.4025,1.0,0.646199
7,9,3.75,6.00,1.65,4.00,5.00,2,2,0.266667,0.166667,...,0.500000,1,-0.25,1.00,-0.35,14.0625,36.0000,2.7225,1.0,0.715213
8,11,4.75,8.00,1.40,4.75,7.00,3,2,0.210526,0.125000,...,0.333333,1,0.00,1.00,-1.60,22.5625,64.0000,1.9600,1.0,0.903607
9,12,4.20,7.50,1.57,4.00,6.00,1,2,0.238095,0.133333,...,1.000000,1,0.20,1.50,0.57,17.6400,56.2500,2.4649,1.0,0.291517


In [27]:
a=0
t=0.8
r=0
w=0
for index,row in combos.iterrows():
    a=a+1
    if row["predict"]>t and row["result1,0"]==1:
        r=r+1
    elif row["predict"]<=t and row["result1,0"]==0:
        r=r+1
    else:
        w=w+1
print("right and wrong",r/a)
        
        
    

right and wrong 0.7420701168614358


In [31]:
#计算准确率
r=0
a=0
for index,row in combos.iterrows():
    if row["predict"]>0.9:
        a=a+1
        if row["result1,0"]==1:
            r=r+1
ra=r/a
ra    

0.9204301075268817

In [32]:
a

465

In [12]:
r/a

0.7420701168614358